<a href="https://colab.research.google.com/github/sridas123/CS6320/blob/master/Nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This snippet encodes the string labels with integers, feature vectors with integers from the vocabulary

In [0]:
"""encode the labels with integers"""
def label_encode(string):
    labeltrain=[]
    for i in range(0,len(string)):
       if string[i]=="TRUE":
          labeltrain.append(1)
       else:
          labeltrain.append(0)   
    return labeltrain 

    """Integer encode a text"""
def integer_encode(string,vocab_dict,out_of_bag_idx):
    trains=[] 
    for i in range(0, len(string)):
        ptrain=[]
        for word in string[i]:
            if word in vocab_dict.keys():
               ptrain.append(vocab_dict[word])
            else:
               ptrain.append(vocab_dict[word+'s'])
        ptrain.append(out_of_bag_idx)       
        trains.append(ptrain)      
    return trains   

def pad_sequence_str(string,maxlength):  
    ostring=pad_sequences(string,padding='post',maxlen=maxlength)
    return ostring

This snippet populates the vocabulary with unique words and their ids

In [0]:
"""Populate the vocabulary"""  
def populate_vocab(ptrain,vocab_dict,start=0):
    
    for i in range(0,len(ptrain)):
        str_list=ptrain[i]
        for word in str_list:
            #print (word)
            if (word in vocab_dict.keys()) or (word+'s' in vocab_dict.keys()):
               continue  
            else:
               start=start+1
               vocab_dict[word]=start   
    return vocab_dict,start 

"""Calculate the value for the out of bag words"""
def populate_vocab_test(ptest,vocab_dict,start=0):  

    for i in range(0,len(ptest)):
        str_list=ptest[i]
        for word in str_list:
            if (word not in vocab_dict) and (word+'s' not in vocab_dict):
                vocab_dict[word]=start   
    return vocab_dict             

This snippet parses the text and cleans the string





In [0]:
def clean_string(string):
    #print ("I am cleaning the string")  
    string=string.replace(".", "")
    string=string.replace("\n", "")
    string=string.replace("(", " ")
    string=string.replace(")", " ")
    string=string.replace("[", " ")
    string=string.replace("]", " ")
    string=string.replace("'", "")
    string=string.replace('"', "")
    string=string.replace("-","")
    string=string.replace(",","")
    string=string.replace(";","")
    string=string.lower()
    #strip spaces from beg and end
    string=string.strip()
    return string    

def parsexml(filename):
  p=[]
  h=[]
  labels=[] 
  response = urllib.request.urlopen(filename).read()
  root = ET.fromstring(response)
  for premise in root.findall('pair'):
      #print ("I am SRijita")
      ptext = premise.find('t').text
      htext = premise.find('h').text
      label= premise.get('value')
      """Cleaning the text"""
      ptext_clean=clean_string(ptext)
      htext_clean=clean_string(htext)
      """Finding the tokens and storing them in array"""
      ptext_clean_tokens=ptext_clean.split()
      #print (ptext_clean_tokens)
      htext_clean_tokens=htext_clean.split()
      p.append(ptext_clean_tokens)
      h.append(htext_clean_tokens)
      labels.append(label)
  #print (p[0])    
  return p,h,labels   

Main module to run the code

In [127]:
from __future__ import division
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation,Bidirectional,Embedding,LSTM
from copy import deepcopy
from random import randint
import numpy as np
import tensorflow as tf
import ssl
import urllib.request
import xml.etree.ElementTree as ET
#out_of_bag_idx=50000
BATCH_SIZE=10

trainfile="https://www.hlt.utdallas.edu/~moldovan/CS6320.20S/train.xml"
testfile="https://www.hlt.utdallas.edu/~moldovan/CS6320.20S/test.xml"
ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == "__main__":
   vocab_dict={}
   p_train,h_train,labels_train=parsexml(trainfile)
   p_test,h_test,labels_test=parsexml(testfile) 
   vocab_dict,offset=populate_vocab(p_train,vocab_dict)
   vocab_dict,offset=populate_vocab(h_train,vocab_dict,offset)
   print ("The length of vocab is", len(vocab_dict))
   train_vocab_length=len(vocab_dict)
   vocab_dict=populate_vocab_test(p_test,vocab_dict,offset+1)
   vocab_dict=populate_vocab_test(h_test,vocab_dict,offset+1) 
   print ("The length of vocab is",len(vocab_dict),offset+1)
   #a=[]
   #for key,value in vocab_dict.items():
   #    a.append(value)
   #print (max(a))      
   """Integer encode premise and hypotheses"""
   ptrain=integer_encode(p_train,vocab_dict,offset+1)
   htrain=integer_encode(h_train,vocab_dict,offset+1)
   ptest=integer_encode(p_test,vocab_dict,offset+1)
   htest=integer_encode(h_test,vocab_dict,offset+1)
   #print (ptrain[0:2])
   #print (htrain[0:2])
   """Integer encode the labels"""
   labeltrain=label_encode(labels_train)
   labeltest=label_encode(labels_test)
   """Finds the maxlength among premise and hypothesis"""
   max_premise = max((x) for x in ptrain)
   max_hypothesis=max((x) for x in htrain)
   max_length=max(len(max_premise),len(max_hypothesis))
   """Padding the sequences of premise and hypothesis"""
   ptrain_paded=pad_sequence_str(ptrain,max_length)
   htrain_paded=pad_sequence_str(htrain,max_length)
   ptest_paded=pad_sequence_str(ptest,max_length)
   htest_paded=pad_sequence_str(htest,max_length)
   """One hot encodings of train and test labels"""
   labeltrain=np.array(labeltrain)
   labeltest=np.array(labeltest)
   labeltrain_onehot = to_categorical(labeltrain)
   labeltest_onehot = to_categorical(labeltest)
   #print (labeltrain_onehot[0:5],labeltrain[0:5])
   #print (labeltest_onehot[0:5])

   """Concatenate the features of premise and labels"""
   ptrain_paded=np.array(ptrain_paded)
   htrain_paded=np.array(htrain_paded)
   ptest_paded=np.array(ptest_paded)
   htest_paded=np.array(htest_paded)
   #print ("ptrain_paded",ptrain_paded[5],p_train[5],vocab_dict['food'],vocab_dict['foods'])
   #print ("htrain_paded",htrain_paded.shape)
   #print ("ptest_paded",ptest_paded.shape)
   #print ("htest_paded",htest_paded.shape)
   train_feat=np.hstack((ptrain_paded,htrain_paded))
   test_feat=np.hstack((ptest_paded,htest_paded))
   print ("The shape of the train features", train_feat.shape)
   print ("The shape of the test features", test_feat.shape)

The length of vocab is 4657
The length of vocab is 7722 4658
The shape of the train features (567, 54)
The shape of the test features (800, 54)


Deep learning part of the project

In [128]:
BUFFER_SIZE=train_feat.shape[0]
"""Create tensors from data"""
train_tensor=tf.data.Dataset.from_tensor_slices((train_feat,labeltrain_onehot))
print (train_tensor.element_spec)
test_tensor=tf.data.Dataset.from_tensor_slices((test_feat,labeltest_onehot))
print (test_tensor.element_spec)
train_batch = (train_tensor.shuffle(BUFFER_SIZE).batch(BATCH_SIZE))
"""No shuffling for test data set"""
test_batch = (test_tensor.batch(BATCH_SIZE))

(TensorSpec(shape=(54,), dtype=tf.int32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))
(TensorSpec(shape=(54,), dtype=tf.int32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))


In [129]:
import time
"""Keras model built"""
model = Sequential()
model.add(Embedding(input_dim=train_vocab_length+2,output_dim=25))
model.add(Bidirectional(LSTM(25)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
# Train model on your dataset
model.fit(train_batch,epochs=5)
start_time=time.time()
score = model.evaluate(test_batch)
print ("The throughput for inference time in seconds %",  (time.time() - start_time))
print ("The score is",score)
ypred=model.predict_classes(test_batch)
#print (ypred[0:20])

Epoch 1/5
57/57 [==============================] - 1s 12ms/step - loss: 0.6933 - accuracy: 0.5044
Epoch 2/5
57/57 [==============================] - 1s 12ms/step - loss: 0.6759 - accuracy: 0.7443
Epoch 3/5
57/57 [==============================] - 1s 12ms/step - loss: 0.4780 - accuracy: 0.8201
Epoch 4/5
57/57 [==============================] - 1s 13ms/step - loss: 0.1965 - accuracy: 0.9400
Epoch 5/5
80/80 [==============================] - 0s 6ms/step - loss: 1.3032 - accuracy: 0.5200
The throughput for inference time in seconds % 1.2267887592315674
The score is [1.3031561374664307, 0.5199999809265137]
[0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1]


Code to print the metrics; Accuracy, Recall,F1-score

In [130]:
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

print ("The accuracy is",accuracy_score(labeltest, ypred))
print ("The Recall is",recall_score(labeltest, ypred))
print ("The f1-score is",f1_score(labeltest, ypred))
print ("The precision-score is",precision_score(labeltest, ypred))

The accuracy is 0.52
The Recall is 0.69
The f1-score is 0.5897435897435898
The precision-score is 0.5149253731343284


Save the learnt model

In [0]:
#Saving the model
model_json = model.to_json()
with open("model.json", "w") as ofile:
    ofile.write(model_json)
model.save_weights("model.h5")